<a href="https://colab.research.google.com/github/FuzzysTodd/1inchProtocol/blob/master/Chrome_Bridge_and_Macro_Controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# A Python script acting as the "blink computator" bridge between MQL5 and Chrome.
#
# 1. Runs a simple web server (Flask) to listen for POST requests from the MQL5 EA.
# 2. Uses the PyChromeDevTools library (CDP) to execute macros in Chrome.
#
# Requires: pip install flask PyChromeDevTools
#
# NOTE: You MUST launch Chrome with remote debugging enabled:
# chrome.exe --remote-debugging-port=9222 --remote-allow-origins=*

import json
from flask import Flask, request, jsonify
import PyChromeDevTools
import time
import threading

app = Flask(__name__)
# Global variable for the Chrome interface (assuming Chrome is running on 9222)
chrome = None

# --- MACRO FUNCTIONS ---
def run_chrome_macro(action, symbol, error_code):
    """
    Executes a specific automation sequence in Chrome based on the error received.
    """
    global chrome

    # Simple retry logic to ensure Chrome connection is available
    if chrome is None:
        try:
            # Connect to Chrome running on localhost:9222
            chrome = PyChromeDevTools.ChromeInterface(host="127.0.0.1", port=9222)
            # Enable the domains we need
            chrome.Page.enable()
            chrome.Network.enable()
            print("Chrome DevTools Protocol (CDP) connection established.")
        except Exception as e:
            print(f"ERROR: Could not connect to Chrome DevTools (port 9222). Ensure Chrome is running with --remote-debugging-port=9222. Error: {e}")
            return False

    try:
        if action == "SERVER_ERROR_RELOAD_PAGE":
            print(f"MACRO: Server Error detected ({error_code}). Reloading the chart web page.")
            # Example macro: Reload the specific tab where the chart/issue is.
            # We assume the relevant page is the main page (index 0)

            # Navigate to a fresh page (e.g., your broker's site, or a simple status page)
            # You would replace this URL with the page that is causing issues
            chrome.Page.navigate(url="https://www.google.com/search?q=MQL5+trading+error+fix")
            # Wait for the load event to fire
            chrome.wait_event("Page.loadEventFired", timeout=10)

            print("MACRO COMPLETE: Page reloaded successfully.")

        elif action == "MQL_RUNTIME_FIX_PAGE":
            print(f"MACRO: MQL Runtime Error detected ({error_code}). Executing complex script fix.")
            # Example macro: Execute JavaScript to fix a UI issue

            # The script attempts to find a common class name and change its style or content.
            # You would customize this to target your specific "Chrome issues."
            js_script = """
            (function() {
                const element = document.querySelector('.problematic-div-class');
                if (element) {
                    element.style.backgroundColor = 'lightgreen';
                    element.textContent = 'MQL Error Fixed!';
                    return 'Success';
                }
                return 'Element not found';
            })();
            """
            result, _ = chrome.Runtime.evaluate(expression=js_script)

            # result structure is complex, typically check result['result']['value']
            print(f"MACRO COMPLETE: JS Script executed. Result: {result.get('result', {}).get('value')}")

        else:
            print(f"MACRO WARNING: Unknown action received: {action}")
            return False

        return True

    except Exception as e:
        print(f"CDP Execution Error: {e}")
        # Reset chrome to force a reconnect attempt next time
        chrome = None
        return False

# --- FLASK ENDPOINT ---
@app.route('/api/macro', methods=['POST'])
def handle_macro_request():
    """
    Receives the POST request from the MQL5 EA.
    """
    if request.is_json:
        data = request.get_json()
        action = data.get('action', 'NO_ACTION')
        symbol = data.get('symbol', 'UNKNOWN')
        error_code = data.get('error_code', 0)

        print("-" * 40)
        print(f"RECEIVED REQUEST from MQL5:")
        print(f"  Action: {action}")
        print(f"  Symbol: {symbol}")
        print(f"  Error Code: {error_code}")

        # Run the macro in a separate thread to ensure the HTTP response
        # is returned immediately to MQL5, preventing MT5 from waiting too long.
        macro_thread = threading.Thread(target=run_chrome_macro, args=(action, symbol, error_code))
        macro_thread.start()

        return jsonify({"status": "Macro triggered successfully", "received_action": action}), 200
    else:
        return jsonify({"status": "Failed", "error": "Content-Type must be application/json"}), 400

# --- MAIN EXECUTION ---
if __name__ == '__main__':
    print("--- Starting MQL5-to-Chrome Bridge (Python Flask Server) ---")
    print(f"Listening on: http://127.0.0.1:5000/api/macro")
    print("Please ensure Chrome is running with the following flag:")
    print("  chrome.exe --remote-debugging-port=9222 --remote-allow-origins=*")
    print("-" * 55)

    # Run the Flask server
    app.run(host='127.0.0.1', port=5000)

ModuleNotFoundError: No module named 'PyChromeDevTools'

# New Section

In [ ]:
!pip install PyChromeDevTools